* ## 创建线程实例的两种方法
  1. ### 创建一个Thred实例并为它提供期望可执行调用对象
  2. ### 通过继承Thread类，重写它的run方法


In [5]:
import time
from threading import Thread

# 1 
def count_down(n):
    while n >0:
        time.sleep(2)
        print(f'count_down  {n}')
        n -= 1

f = Thread(target=count_down, args=(5,))
f.start()

# 2
class CountDown(Thread):
    def __init__(self, n):
        super().__init__()
        self.n = n
    
    def run(self):
        while self.n > 0:
            print('count', self.n)
            self.n -= 1
            time.sleep(1)
            
c = CountDown(5)
c.start()


count 5
count 4
count_down  5
count 3
count 2
count_down  4
count 1
count_down  3
count_down  2
count_down  1



# [Event object](https://docs.python.org/3/library/threading.html#event-objects)


* 某线程发送一个event信号,另一线程等待此event,这是threads之间通信的最简单的机制

* event对象管理一个内部标识，set()方法可设置标识为true，clear()方法则设置标识为false，wait()的调用会阻塞当前线程，直到标识为true

* ## class threading.Event
  此类实现了Event object，初始标识为false
  * is_set()
  ```
  如果内部标识为true 返回true
  ```

  * set()
  ```
  设置内部标识为true，唤醒所有正在等待该event的线程，如果标识为true时，调用wait()方法不会阻塞
  ```

  * clear()
  ```
  重新设置内部标识为false，之后的线程调用wait()会阻塞，直到set()再次调用
  ```

  * wait(timeout = None)
  ```
  阻塞线程直到标识变为true。如标识为真立即返回，否则阻塞直到其他线程set()的调用,或超时
  ```
  ```
  当向wait传入非none的timeout参数，参数应为浮点，单位为秒
  ```
  


In [11]:
import time
from threading import Thread, Event
def count():
    global counter, event
    print('waiting event signal')
    event.wait(timeout=5)
    print('thread continue')
    for i in range(10):
        print(counter)
        counter += 1


counter = 0
event = Event()
t1 = Thread(target=count, args=())
print('start thread')
t1.start()
time.sleep(2)
print('set event')
event.set()

start thread
waiting event signal
thread continue
0
1
2
3
4
5
6
7
8
9
set event


# [Lock Object ](https://docs.python.org/3/library/threading.html#lock-objects)

* 锁是最原始的同步基元,其不属于任何线程，在python中锁时最底层的可用的同步基元，通过_thread的扩展模块直接实现
* 锁处于两个状态，锁和未锁

* ## class threading.Lock
  这个类实现了一个lock对象，线程一但被锁，随后任意的对它的访问都会阻塞，直到锁的释放
  任何线程都可以释放它

  * acquire(block=True, timeout=-1)
  ```
  获取一个锁，阻塞或非阻塞
  当调用时blocking=True(默认)，阻塞直到锁释放，获取锁后设置为锁定返回true
  当调用时blocking=False不阻塞，立即返回false
  ```
  * release()
  ```
  释放锁，任意线程可调用此方法
  当锁时加锁状态，重置为未锁，并返回，如果有多个线程阻塞等待锁释放，只允许他们中的一个   继续
  ```

In [12]:
import time
from threading import Thread, Lock
from functools import wraps


def timer(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        start = time.time()
        res = func(*args,**kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return res
    return wrapper


number = 0
number_lock = 0


@timer
def count():
    global number
    for i in range(1000000):
        number += 1


@timer
def count_with_lock():
    global number_lock,lock
    for i in range(1000000):
        with lock:
            number_lock += 1


lock = Lock()
t1 = Thread(target=count,args=())
t2 = Thread(target=count,args=())
t3 = Thread(target=count_with_lock,args=())
t4 = Thread(target=count_with_lock,args=())
t1.start()
t2.start()
t3.start()
t4.start()
t1.join()
t2.join()
t3.join()
t4.join()
print(number)
print(number_lock)

count 0.17389845848083496
count 0.20389485359191895
count_with_lockcount_with_lock 1.4451658725738525 1.4291746616363525

1253794
2000000


# [Condition Objects](https://docs.python.org/3/library/threading.html#condition-objects)

  ```
  其他方法 必须在持有锁的情况下访问，wait()释放锁，并阻塞直到其他线程通过notify()或notify_all()唤醒，一旦唤醒，wait()会重新获取锁并返回

  notify()方法唤醒一个正在等待条件满足的线程，notify_all()唤醒所有

  note：notify() 和 notify_all() 方法并不会立即释放锁，也就是说被唤醒的线程的wait()不会立即返回值，只有当调用notify 方法的线程最终释放掉锁的时候，wait()才会返回值

  通过锁来同步获取某些共享状态是条件变量的典型编程方式，线程反复通过调用wait()监听某些变化直到获取到需要的状态，当某些进程修改了状态可能会满足等待者的需求,就会调用notify()或notify_all()
  ```
  ```python
  # consume
  with cv:
    while not an_item_is_available():
        cv.wait()
    get_an_available_item()

  # produce
  with cv:
    make_an_item_available()
    cv.notify()
  ```

# [Semaphore Objects](https://docs.python.org/3/library/threading.html#semaphore-objects)
 
```
semaphore对象内部管理一个计数器，每次acquire()方法调用会-1，每次release()调用会增加，
计数器不会小于0，当计数器为0时，acquire()的调用会阻塞，直到release()调用
```
* class threading.Semaphore(value=1)
  ```
  可选参数给了内部counter初始值，默认为1，如果传入参数小于0，ValueError
  ```

  * acquire(blocking=True,timeout=none)
  ```
  不传参调用
  如果开始counter大于0，减一后立即返回
  如果开始counter等于0，保持阻塞直到release()被调用，一旦唤醒立即减一并返回，每次release()的调用智慧唤醒一个线程，线程唤醒的顺序不可信
  ```
  * release()
  ```
  释放一个semaphore,内部counter+1
  ```